In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
from tqdm import tqdm
from io import StringIO
import time

In [2]:
master = pd.read_csv("C:/Users/erknud3/fpl-optimization/model/FPL-ID-MAP/master.csv")
master = master[['code','first_name', 'second_name', 'web_name', 'fbref', 'transfermarkt', '24-25']]
master = master.dropna(subset=['fbref', '24-25'])


master['fbref'] = master['fbref'].astype(str)
master['transfermarkt'] = master['transfermarkt'].fillna(0).astype(int)
master['24-25'] = master['24-25'].astype(int)

In [3]:
master.sort_values(by='24-25', ascending=True)

,code,first_name,second_name,web_name,fbref,transfermarkt,24-25
1784,438098,Fábio,Ferreira Vieira,Fábio Vieira,6412cc03,537598,1
1315,205651,Gabriel,Fernando de Jesus,G.Jesus,b66315ae,363205,2
1525,226597,Gabriel,dos Santos Magalhães,Gabriel,67ac5bb8,435338,3
1439,219847,Kai,Havertz,Havertz,fed7cb61,309400,4
1966,463748,Karl,Hein,Hein,aa81d8f8,493513,5
...,...,...,...,...,...,...,...
2287,518438,Kaelan,Casey,Casey,cd4b2c5f,845321,643
2420,574398,Franco,Umeh-Chibueze,Umeh-Chibueze,8e526344,943560,644
2311,530318,Kaden,Rodney,Rodney,49375cdd,860078,647
2422,575034,Ethan,Wheatley,Wheatley,5c368c88,888639,648


In [4]:
len(master['fbref'].unique())

640

In [5]:
unique_ids = master['fbref'].unique()
seasons = ['2021-2022', '2022-2023', '2023-2024', '2024-2025']

fbref_urls = []

for fbref in unique_ids:
    for season in seasons:
        url = f"https://fbref.com/en/players/{fbref}/matchlogs/{season}/"
        fbref_urls.append(url)

In [6]:
# Lists to track successful and failed URLs
successful_urls = []
failed_urls = []

# Folder to save HTML files
output_folder = "C:/Users/erknud3/fpl-optimization/model/html_files/matchlogs_players"
os.makedirs(output_folder, exist_ok=True)

# Loop through URLs with progress bar
with tqdm(total=len(fbref_urls), desc="Processing URLs") as pbar:
    for url in fbref_urls:
        try:
            # Extract fbref ID and season for naming the file
            parts = url.split('/')
            fbref_id = parts[5]
            season = parts[7]

            # Send a request to fetch the HTML content
            response = requests.get(url)
            response.raise_for_status()  # Raises an HTTPError for bad responses

            # Create a filename based on fbref ID and season
            filename = f"{fbref_id}_matchlogs_{season}.txt"
            file_path = os.path.join(output_folder, filename)

            # Save the HTML content to the file
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)

            # Append to successful_urls
            successful_urls.append(url)

        except requests.RequestException as e:
            # Append to failed_urls in case of an error
            failed_urls.append(url)
            print(f"Failed to process URL: {url}. Error: {e}")
            
        time.sleep(6)

        # Update the progress bar
        pbar.update(1)

# Save failed URLs for future reference
if failed_urls:
    with open(os.path.join(output_folder, 'failed_urls.txt'), 'w') as f:
        for url in failed_urls:
            f.write(url + '\n')

# Save successful URLs for future reference
if successful_urls:
    with open(os.path.join(output_folder, 'successful_urls.txt'), 'w') as f:
        for url in successful_urls:
            f.write(url + '\n')

print("Processing complete.")


Processing URLs:   0%|          | 0/2560 [00:00<?, ?it/s]

Failed to process URL: https://fbref.com/en/players/2f90f6b8/matchlogs/2021-2022/. Error: 429 Client Error: Too Many Requests for url: https://fbref.com/en/players/2f90f6b8/matchlogs/2021-2022/


Processing URLs:   0%|          | 0/2560 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Folder containing the saved URLs and HTML files
output_folder = "C:/Users/erknud3/fpl-optimization/model/html_files/matchlogs_players"

# Load the failed URLs from the previous run
failed_urls = []
failed_urls_path = os.path.join(output_folder, 'failed_urls.txt')
if os.path.exists(failed_urls_path):
    with open(failed_urls_path, 'r') as f:
        failed_urls = [line.strip() for line in f]

# Lists to track the new successful and failed URLs
successful_urls_retry = []
failed_urls_retry = []

# Retry the process for failed URLs
with tqdm(total=len(failed_urls), desc="Retrying Failed URLs") as pbar:
    for url in failed_urls:
        try:
            # Extract fbref ID and season for naming the file
            parts = url.split('/')
            fbref_id = parts[5]
            season = parts[7]

            # Send a request to fetch the HTML content
            response = requests.get(url)
            response.raise_for_status()  # Raises an HTTPError for bad responses

            # Create a filename based on fbref ID and season
            filename = f"{fbref_id}_matchlogs_{season}.txt"
            file_path = os.path.join(output_folder, filename)

            # Save the HTML content to the file
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)

            # Append to successful_urls_retry
            successful_urls_retry.append(url)

        except requests.RequestException as e:
            # Append to failed_urls_retry in case of an error
            failed_urls_retry.append(url)
            print(f"Failed to process URL: {url}. Error: {e}")
            
        time.sleep(6)

        # Update the progress bar
        pbar.update(1)

# Save the new failed URLs for future reference
if failed_urls_retry:
    with open(os.path.join(output_folder, 'failed_urls_retry.txt'), 'w') as f:
        for url in failed_urls_retry:
            f.write(url + '\n')

# Save the new successful URLs for reference
if successful_urls_retry:
    with open(os.path.join(output_folder, 'successful_urls_retry.txt'), 'w') as f:
        for url in successful_urls_retry:
            f.write(url + '\n')

print("Retry processing complete.")


In [14]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0"}

In [ ]:
all_dfs = []
with tqdm(total=len(fbref_urls), desc="Processing URLs") as pbar:
    for url in fbref_urls:
        try:
            with requests.Session() as session:
                data = session.get(url, headers=headers)
                soup = BeautifulSoup(data.text, "html.parser")
                ids = re.search(r"/players/([a-zA-Z0-9]{8})/", url)
                season = re.search(r"/(\d{4}-\d{4})/", url)
                squad_id = table.find_all("td", {"data-stat": "team"})[0].find('a')['href'].split('/en/squads/')[1].split('/')[0]

                table = soup.find("table", {"id": "matchlogs_all"})

                df = pd.read_html(StringIO(str(table)))[0]
                df.columns = [
                    f"{i} {j}" if "Unnamed" not in i else j for i, j in df.columns
                ]
                
                df["fbref"] = ids.group(1)
                df["season"] = season.group(1)
                df["squad_id"] = squad_id
                
                all_dfs.append(df)
                
                time.sleep(6)
                
        except Exception as e:
            print(f"An error occurred for URL: {url}: {e}")

        pbar.update(1)
        #print(f"Status code: {data.status_code}")

In [46]:
match_logs = pd.concat(all_dfs)

match_logs = match_logs.rename ({
    "Performance Gls": "Goals", "Performance Ast": "Assists", "Performance PK": "PKs", "Performance PKatt": "PKatt", "Performance CrdY": "CrdY", "Performance CrdR": "CrdR", "Expected xG": "xG", "Expected npxG": "npxG", "Expected xAG": "xAG",
    }, axis=1)
match_logs.dropna(inplace=True)
match_logs = match_logs.merge(master[['fbref', 'transfermarkt', '24-25', 'first_name', 'second_name', 'web_name']], on='fbref', how='inner')
match_logs = match_logs.rename({'24-25': 'fpl_id'}, axis=1)
match_logs.columns = match_logs.columns.str.lower()
match_logs = match_logs[['date', 'fbref', 'transfermarkt', 'fpl_id', 'first_name', 'second_name', 'web_name', 'season', 'comp', 'round', 'venue', 'squad_id', 'squad', 'opponent', 'start', 'min', 'goals', 'assists', 'pks', 'pkatt', 'xg', 'npxg', 'xag']]
match_logs[['min', 'goals', 'assists', 'pks', 'pkatt', 'xg', 'npxg', 'xag']] = match_logs[['min', 'goals', 'assists', 'pks', 'pkatt', 'xg', 'npxg', 'xag']].replace("On matchday squad, but did not play", 0)
match_logs['squad'] = match_logs['squad'].str.replace(r'\b[a-z]+\b', '', regex=True).str.strip()
match_logs['opponent'] = match_logs['opponent'].str.replace(r'\b[a-z]+\b', '', regex=True).str.strip()

In [54]:
unique_squad_ids = match_logs['squad_id'].unique()
unique_squad_names = match_logs['squad'].unique()
seasons = ['2021-2022', '2022-2023', '2023-2024', '2024-2025']

squad_urls = []

for squad_id, squad_name in zip(unique_squad_ids, unique_squad_names):
    for season in seasons:
        url = f"https://fbref.com/en/squads/{squad_id}/{season}/matchlogs/all_comps/shooting/{squad_name.replace(' ', '-')}-Scores-and-Fixtures-All-Competitions"
        squad_urls.append(url)

In [ ]:
squad_urls

In [ ]:
url_squad_data = list(zip(squad_urls, match_logs['squad_id'], match_logs['squad']))

squad_dfs = []
with tqdm(total=len(url_squad_data), desc="Processing URLs") as pbar:
    for url, squad_id, squad_name in url_squad_data:
        try:
            with requests.Session() as session:
                data = session.get(url, headers=headers)
                soup = BeautifulSoup(data.text, "html.parser")

                table = soup.find("table", {"id": "matchlogs_for"})

                df = pd.read_html(StringIO(str(table)))[0]
                df.columns = [
                    f"{i} {j}" if "Unnamed" not in i else j for i, j in df.columns
                ]
                
                df['squad_id'] = squad_id
                df['squad_name'] = squad_name
                
                squad_dfs.append(df)
                
                time.sleep(6)
                
        except Exception as e:
            print(f"An error occurred for URL: {url}: {e}")

        pbar.update(1)

In [64]:
match_logs_squads = pd.concat(squad_dfs)
match_logs_squads = match_logs_squads.rename ({
    "Expected xG": "xG", "Expected npxG": "npxG", "Expected npxG/Sh": "npxG/Sh",
    }, axis=1)
match_logs_squads.dropna(inplace=True)
match_logs_squads.columns = match_logs_squads.columns.str.lower()
#match_logs_squads = match_logs_squads[['date', 'squad_id', 'squad', 'comp', 'round', 'venue', 'result', 'gf', 'ga', 'opponent', 'xg', 'npxg', 'npxg/sh']]
match_logs_squads['squad_name'] = match_logs_squads['squad_name'].str.replace(r'\b[a-z]+\b', '', regex=True).str.strip()
#match_logs_squads['opponent'] = match_logs_squads['opponent'].str.replace(r'\b[a-z]+\b', '', regex=True).str.strip()


In [ ]:
match_logs_squads.keys()